## Task:

#### Write a Monte Carlo radiation transfer code to simulate emission from an isotropic point source at the origin of a uniform density sphere. Assume the Monte Carlo photon packets are scattered isotropically on their random walks through the sphere. For the case where the scattering albedo is unity, compute as a function of the sphere’s radial optical depth, $\tau_r$, the average number of scatterings per Monte Carlo packet, $<N_{scatt}>$. Plot $<N_{scatt}>$ versus $t_r$ and compare this with the analytic approximation $<N_{scatt}> \approx \tau_r + \tau_r^2/2$, derived in the lecture notes. Investigate radial optical depths from 0.1 up to 200. To ensure that you have a working MCRT code, please send me a figure comparing your MCRT results with the analytic expression.

## Code

In [ ]:
program uni_sphere
    implicit none
    real :: x,y,z,nx,ny,nz,tau,tau_r_min,tau_r_max,tau_r,r,rmax,E,L,av_scat,num_tau_step
    integer :: npackets,nscat,num_tau_r,i,j,k
    real, allocatable, dimension(:) :: taus(:),av_scatters(:)
    npackets = 1000
    tau_r_min = 0.1
    tau_r_max = 200
    num_tau_r = 200
    num_tau_step = (REAL(tau_r_max) - REAL(tau_r_min))/(num_tau_r-1)  !100 elements
    allocate(taus(num_tau_r))
    allocate(av_scatters(num_tau_r))
    do k = 1,num_tau_r
        taus(k) = 0.1 + num_tau_step*(k-1)
    end do
    rmax = 1
    open(unit=1,file='testl3b.txt')
    do j = 1,num_tau_r
        tau_r = taus(j)
        nscat = 0
        do i = 1, npackets
            call emit_packet(x,y,z,nx,ny,nz)
            r = x*x + y*y + z*z
            do while (r <= rmax) ! Packet is in sphere. Do we even need r>=0?
                call random_number(E)
                tau = -log(E)
                L = tau * rmax / tau_r ! Distance travelled
                ! update packet position, x,y,z
                x = x + L * nx
                y = y + L * ny
                z = z + L * nz
                r = x*x + y*y + z*z
                if (r >= rmax) EXIT
                nscat = nscat + 1
                call scatter(nx,ny,nz)
            end do
        end do
        av_scat = nscat/npackets
        av_scatters(j) = av_scat
        write(1,*) tau_r,av_scatters(j)
    end do
    close(1)
end program uni_sphere

subroutine emit_packet(x,y,z,nx,ny,nz)
    implicit none
    real, parameter :: pi = 3.14159
    real :: E1,E2,theta,phi
    real, intent(out) :: x,y,z,nx,ny,nz
    x=0
    y=0
    z=0
    call random_number(E1)
    call random_number(E2)
    theta = ACOS(2*E1 - 1)
    phi = 2*pi*E2
    nx = SIN(theta)*COS(phi)
    ny = SIN(theta)*SIN(phi)
    nz = COS(theta)
end subroutine emit_packet

subroutine scatter(nxs,nys,nzs)
    implicit none
    real, parameter :: pi = 3.14159
    real :: E1,E2,theta,phi
    real, intent(out) :: nxs,nys,nzs
    call random_number(E1)
    call random_number(E2)
    theta = ACOS(2*E1 - 1)
    phi = 2*pi*E2
    nxs = SIN(theta)*COS(phi)
    nys = SIN(theta)*SIN(phi)
    nzs = COS(theta)
end subroutine scatter